In [ ]:
import zipfile
import os

# Path ke file ZIP dataset
dataset_zip_path = '/content/datasetklasifikasi.zip'

# Path direktori ekstraksi
dataset_extract_path = '/content/datasetklasifikasi'

# Mengekstrak file ZIP
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_extract_path)

print(f"Dataset telah diekstrak ke: {dataset_extract_path}")


Dataset telah diekstrak ke: /content/datasetklasifikasi


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import os
from google.colab import files
from PIL import Image

# Path dataset
base_dir = "/content/datasetklasifikasi/datasetklasifikasi"

# Data augmentation & preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load dataset
train_data = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Build Custom CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training model
epochs = 10
history = model.fit(train_data, epochs=epochs)


Found 3968 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 571s 5s/step - accuracy: 0.6626 - loss: 2.6913
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 621s 5s/step - accuracy: 0.8243 - loss: 0.4495
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 569s 5s/step - accuracy: 0.8404 - loss: 0.4562
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 563s 5s/step - accuracy: 0.8578 - loss: 0.3935
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 575s 5s/step - accuracy: 0.8732 - loss: 0.3635
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 570s 5s/step - accuracy: 0.8599 - loss: 0.3827
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 568s 5s/step - accuracy: 0.8883 - loss: 0.3405
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 562s 5s/step - accuracy: 0.9042 - loss: 0.3087
Epoch 9/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 627s 5s/step - accuracy: 0.8923 - loss: 0.3172
Epoch 10/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 566s 5s/step - accuracy: 0.8952 - loss: 0.3019


In [ ]:
# Function to predict an uploaded image
def predict_image():
    uploaded = files.upload()
    for file_name in uploaded.keys():
        # Change target_size to (224, 224)
        img = load_img(file_name, target_size=(224, 224))
        img_array = img_to_array(img) / 255.0
        # Reshape to (1, 224, 224, 3) for batch prediction
        img_array = img_array.reshape((1, 224, 224, 3))

        prediction = model.predict(img_array)[0][0]
        label = "Sapi" if prediction > 0.5 else "Bukan Sapi"

        img.show()
        print(f"Prediksi: {label} ({prediction:.2f})")

# Call the function to upload and predict
predict_image()

Saving learning.jpeg to learning.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Prediksi: Bukan Sapi (0.12)


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

from google.colab import files
files.download('model.tflite')


Saved artifact at '/tmp/tmp8ttz2arx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132944119081744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119082512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119083856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119081936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119083280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119084624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119082128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119085008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119085392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132944119086736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13294411908654

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>